Handling imbalanced data in customer churn prediction

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/14_imbalanced/customer_churn.csv")
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6190,2867-UIMSS,Male,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Electronic check,80.50,80.5,Yes
6018,8859-DZTGQ,Male,0,No,No,33,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.35,689.75,No
2296,4706-DGAHW,Male,1,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,No,Electronic check,45.20,45.2,No
1859,4143-HHPMK,Male,0,No,No,52,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.35,4338.6,Yes
5965,5975-BAICR,Male,0,Yes,Yes,36,Yes,No,DSL,Yes,...,Yes,No,No,Yes,One year,Yes,Credit card (automatic),63.70,2188.5,No


In [4]:

df.drop('customerID',axis='columns',inplace=True)

In [5]:
df1 = df[df.TotalCharges!=' ']
df1.shape

(7032, 20)

In [6]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_12084\973151263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.TotalCharges = pd.to_numeric(df1.TotalCharges)


In [7]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_12084\2045096646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace('No internet service','No',inplace=True)
C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_12084\2045096646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace('No phone service','No',inplace=True)


In [8]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_12084\1648037665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col].replace({'Yes': 1,'No': 0},inplace=True)


In [9]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_12084\698335744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['gender'].replace({'Female':1,'Male':0},inplace=True)


In [10]:
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [11]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [12]:
X = df2.drop('Churn',axis='columns')
y = testLabels = df2.Churn.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()

0.0    4130
1.0    1495
Name: Churn, dtype: int64

Build an Logistic regression model

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix , classification_report

In [16]:
def log_reg(X_train, y_train, X_test, y_test, weights):
    
    if weights == -1:
        model = LogisticRegression()
    else:
        model = LogisticRegression(class_weight={0:weights[0], 1:weights[1]})
    
    model.fit(X_train, y_train)
    acc= model.score(X_test, y_test)
    print('Accuracy', acc, '\n')
    
    y_pred = model.predict(X_test)
    print('Predictions :', y_pred[:5], '\n')
    
    print(classification_report(y_test, y_pred))
    return y_pred

In [17]:
y_preds = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.8031272210376688 

Predictions : [0. 0. 0. 0. 1.] 

              precision    recall  f1-score   support

         0.0       0.84      0.90      0.87      1033
         1.0       0.66      0.54      0.59       374

    accuracy                           0.80      1407
   macro avg       0.75      0.72      0.73      1407
weighted avg       0.79      0.80      0.80      1407



# __Lets handle imbalanced data__

__Method 1: Undersampling__

In [18]:
count_class_0, count_class_1 = df1.Churn.value_counts()

df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [19]:
count_class_0, count_class_1

(5163, 1869)

In [20]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under sampling')
print(df_test_under.Churn.value_counts())

Random under sampling
0    1869
1    1869
Name: Churn, dtype: int64


In [22]:
X = df_test_under.drop('Churn', axis=1)
y = df_test_under['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [23]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [24]:
y_preds = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.7700534759358288 

Predictions : [0 1 1 0 1] 

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       374
           1       0.75      0.81      0.78       374

    accuracy                           0.77       748
   macro avg       0.77      0.77      0.77       748
weighted avg       0.77      0.77      0.77       748



__Method 2 : Oversampling__

In [26]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_1_over, df_class_0], axis=0)

print('Random over sampling')
print(df_test_over.Churn.value_counts())

Random over sampling
1    5163
0    5163
Name: Churn, dtype: int64


In [27]:
X = df_test_over.drop('Churn', axis=1)
y = df_test_over['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [28]:
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [29]:
y_preds = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.7608906098741529 

Predictions : [0 0 1 0 1] 

              precision    recall  f1-score   support

           0       0.78      0.72      0.75      1033
           1       0.74      0.80      0.77      1033

    accuracy                           0.76      2066
   macro avg       0.76      0.76      0.76      2066
weighted avg       0.76      0.76      0.76      2066



__Method 3: SMOTE__

In [30]:
X = df2.drop('Churn', axis=1)
y = df2['Churn']

In [31]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, stratify=y_sm)

In [33]:
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [34]:
y_preds = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.8020329138431752 

Predictions : [0 1 0 0 1] 

              precision    recall  f1-score   support

           0       0.82      0.77      0.80      1033
           1       0.78      0.83      0.81      1033

    accuracy                           0.80      2066
   macro avg       0.80      0.80      0.80      2066
weighted avg       0.80      0.80      0.80      2066



__Method 4: Use of ensamble with undersampling__

In [35]:
df2.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [36]:
X = df2.drop('Churn', axis=1)
y = df2['Churn']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [38]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [39]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [40]:
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
643,1,1,0,0,0.676056,1,0,0,1,1,...,1,0,0,1,0,1,0,0,0,0
6327,0,0,0,0,0.380282,1,0,0,1,0,...,1,0,0,1,0,1,0,0,0,0
6902,1,1,1,0,0.774648,1,1,1,1,1,...,0,0,0,1,0,0,1,0,0,0
4770,1,0,1,1,0.070423,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
4900,0,1,1,0,0.507042,1,1,0,0,0,...,1,0,1,0,0,0,0,1,0,1


In [41]:
df3_class_0 = df3[df3['Churn'] == 0]
df3_class_1 = df3[df3['Churn'] == 1]

In [42]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis=1)
    y_train = df_train.Churn

    return X_train, y_train

In [43]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 0, 1495)

y_pred1 = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.7469793887704336 

Predictions : [0 0 0 1 1] 

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1033
           1       0.52      0.79      0.63       374

    accuracy                           0.75      1407
   macro avg       0.71      0.76      0.72      1407
weighted avg       0.80      0.75      0.76      1407



In [44]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 1495, 2990)

y_pred2 = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.7455579246624022 

Predictions : [0 0 0 0 1] 

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      1033
           1       0.51      0.79      0.62       374

    accuracy                           0.75      1407
   macro avg       0.71      0.76      0.72      1407
weighted avg       0.80      0.75      0.76      1407



In [45]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 2990, 4130)

y_pred3 = log_reg(X_train, y_train, X_test, y_test, -1)

Accuracy 0.7213930348258707 

Predictions : [0 0 0 0 1] 

              precision    recall  f1-score   support

           0       0.92      0.68      0.78      1033
           1       0.49      0.85      0.62       374

    accuracy                           0.72      1407
   macro avg       0.71      0.76      0.70      1407
weighted avg       0.81      0.72      0.74      1407



In [46]:
len(y_pred1)

1407

In [47]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [48]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.91      0.72      0.80      1033
           1       0.51      0.81      0.63       374

    accuracy                           0.74      1407
   macro avg       0.71      0.77      0.72      1407
weighted avg       0.81      0.74      0.76      1407

